## Cat_Classification

In [1]:
# code and data: https://github.com/rpeden/cat-or-not/releases

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from PIL import Image
from random import shuffle, choice
import numpy as np
import os


In [2]:
IMAGE_SIZE = 256
IMAGE_DIRECTORY = './data/training_set'


In [3]:
def label_img(name):
    if name == 'cats': return np.array([1, 0])
    elif name == 'notcats' : return np.array([0, 1])


In [4]:
def load_data():
    print("Loading images...")
    train_data = []
    directories = next(os.walk(IMAGE_DIRECTORY))[1]

    for dirname in directories:
        print("Loading {0}".format(dirname))
        file_names = next(os.walk(os.path.join(IMAGE_DIRECTORY, dirname)))[2]
        for i in range(len(file_names)): # len(file_names)
            image_name = choice(file_names)
            image_path = os.path.join(IMAGE_DIRECTORY, dirname, image_name)
            label = label_img(dirname)
            if "DS_Store" not in image_path:
                img = Image.open(image_path)
                img = img.convert('L')
                img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
                train_data.append([np.array(img)/255, label])
    return train_data


In [5]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation = 'softmax'))

    return model

In [6]:
training_data = load_data()
training_images = np.array([i[0] for i in training_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
training_labels = np.array([i[1] for i in training_data])

Loading images...


StopIteration: 

In [8]:
from sklearn.model_selection import train_test_split

training_images, val_images, training_labels, val_labels = train_test_split(training_images, training_labels, test_size=.2)

In [10]:
training_images.shape, val_images.shape, training_labels.shape, val_labels.shape

((4534, 256, 256, 1), (1134, 256, 256, 1), (4534, 2), (1134, 2))

In [9]:
# validation_split=.2 , shuffle=True 不會打散資料，所以要 train_test_split
# https://github.com/keras-team/keras/issues/4298

print('creating model')
model = create_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print('training model')
model.fit(training_images, training_labels, batch_size=50, epochs=10, verbose=1, validation_data=(val_images, val_labels))
model.save("model.h5")

creating model
training model
Epoch 1/10
91/91 [==============================] - 23s 252ms/step - loss: 0.5910 - accuracy: 0.7647 - val_loss: 0.8738 - val_accuracy: 0.6993
Epoch 2/10
91/91 [==============================] - 23s 250ms/step - loss: 0.3733 - accuracy: 0.8509 - val_loss: 0.6167 - val_accuracy: 0.6993
Epoch 3/10
91/91 [==============================] - 23s 251ms/step - loss: 0.2728 - accuracy: 0.8913 - val_loss: 0.9547 - val_accuracy: 0.6993
Epoch 4/10
91/91 [==============================] - 23s 250ms/step - loss: 0.1816 - accuracy: 0.9292 - val_loss: 1.7434 - val_accuracy: 0.6993
Epoch 5/10
91/91 [==============================] - 23s 253ms/step - loss: 0.1036 - accuracy: 0.9592 - val_loss: 0.6329 - val_accuracy: 0.7381
Epoch 6/10
91/91 [==============================] - 23s 253ms/step - loss: 0.0905 - accuracy: 0.9656 - val_loss: 0.4599 - val_accuracy: 0.8527
Epoch 7/10
91/91 [==============================] - 23s 253ms/step - loss: 0.0793 - accuracy: 0.9709 - val_loss:

In [8]:
IMAGE_DIRECTORY = './data/test_set'
test_data = load_data()
test_images = np.array([i[0] for i in test_data]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)
test_labels = np.array([i[1] for i in test_data])

Loading images...
Loading cats
Loading notcats


In [9]:
test_images.shape

(2665, 256, 256, 1)

In [10]:
test_labels.shape, training_labels.shape

((2665, 2), (5668, 2))

In [11]:
loss, acc = model.evaluate(test_images, test_labels, verbose=1)
print("accuracy: {0}".format(acc * 100))

2665/2665 [==============================] - 7s 3ms/step
accuracy: 63.039398193359375
